In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#runtime
import time
# Base Libraries
import pandas as pd
import matplotlib.pyplot as plt  
import numpy as np
# Transformation
from sklearn.preprocessing import MinMaxScaler
# Models
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import neighbors
# Metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
import numpy as np 
import pandas as pd #data processing, csv file i/o (eg pd.read csv)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from optuna.samplers import TPESampler

#from sklearn.model_selection import GridSearchCV
#Regular Expressions
import re 
# from datetime import datetime

#Visualization
import matplotlib.pyplot as plt
import seaborn as sns 
# import plotly 

import optuna #optimal hyperperameters used in our xgb model
import xgboost as xgb
# from optuna import visualization

#Metrics
from sklearn.metrics import r2_score #r2
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import mean_squared_error #RMSE
from sklearn.metrics import explained_variance_score #EVS

start = time.time()

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))

data = pd.read_csv('../input/hydropower-generation/Hydropower_Consumption.csv', sep=',')
data = data.drop(columns = ["Country"])
data = pd.DataFrame(scaler.fit_transform(data), 
                        columns=['2000','2001','2002','2003','2004','2005',
                                 '2006','2007','2008','2009','2010','2011',
                                 '2012','2013','2014','2015','2016','2017',
                                 '2018','2019'])
data.describe()

In [ ]:
y = data['2019'].copy()
X = data.drop('2019',axis=1).copy()
scaler = StandardScaler()
X = scaler.fit_transform(X)

pd.DataFrame(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.70, random_state=100)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.80, random_state=200)

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
def objective(trial):
      
    learning_rate = trial.suggest_loguniform('learning_rate', 0.04, 0.08)
    max_depth = trial.suggest_int('max_depth', 7,11)
    
    subsample = trial.suggest_loguniform('subsample', 0.6,1.0)
    colsample_bytree = trial.suggest_loguniform('colsample_bytree', 0.9,1.0)
    
    gamma = trial.suggest_loguniform('gamma', 1e-8,1.0)
    max_leaves = trial.suggest_int('max_leaves', 10,200)
    
    reg_alpha = trial.suggest_loguniform ('reg_alpha', 1e-8,6e-5)

    params = {
              
              'booster':'gbtree',
              'objective': 'reg:squarederror',
              'grow_policy': 'lossguide',
              'eval_metric':'rmse',
              'tree_method':'hist',
               

        
              'learning_rate': learning_rate,  
              'max_depth': max_depth,
              'subsample': subsample,
              'colsample_bytree': colsample_bytree, 
              'gamma': gamma,
              'max_leaves': max_leaves,
              
              
              'max_bin': 512,
              'max_delta_step':0,
              'num_parallel_tree':1,
              'scale_pos_weight':1,
              'reg_alpha': reg_alpha, 
              'reg_lambda': 1,
              'min_child_weight':1,
         
              }
    
    

    prune_error = 'eval-' + params['eval_metric']   
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, prune_error)   
    model = xgb.train(params, dtrain, num_boost_round=1000, evals=[(dval, 'eval')],callbacks=[pruning_callback],
                      early_stopping_rounds=10, verbose_eval=0,)
    results = model.eval(dval)
    rmse = np.float(re.search(r'[\d.]+$', results).group(0))
    return rmse

In [ ]:
study = optuna.create_study(sampler=(TPESampler()))
study.optimize(objective, n_trials=100, show_progress_bar=True)



best_params = study.best_params
print(best_params)

cv_output = xgb.cv(best_params, dtrain, num_boost_round=10000, nfold=8,
                   early_stopping_rounds=10, verbose_eval=0, show_stdv=False)
cv_output[['train-rmse-mean', 'test-rmse-mean']].plot()
plt.xlabel('Rounds')
plt.ylabel('rmse')
plt.show()


y_true = np.array(y_test, dtype=np.float)

num_boost_rounds = len(cv_output)
print(num_boost_rounds)

model = xgb.train(dict(best_params), dtrain, num_boost_round= num_boost_rounds,
                  evals=[(dval, 'eval')], early_stopping_rounds=10)


# r2 = r2_score(y_true, y_pred)
MAE = mean_absolute_error(y_test, model.predict(dtest))
# MAPE = np.mean(np.abs((y_true - model.predict(dtest))/y_test)*100)
MBE = np.mean(model.predict(dtest) - y_test)
RMSE = np.sqrt(mean_squared_error(y_test, model.predict(dtest)))
# rRMSE = (RMSE/np.mean(model.predict(dtest)))*100
# n = len(y_test)
# t_stat = np.sqrt(((n-1)*MBE*MBE)/((RMSE*RMSE)-(MBE*MBE)))



print("R^2 in training: %s"  % r2_score(dtrain.get_label(), model.predict(dtrain)))
print("R^2 in testing: %s"  % r2_score(y_test, model.predict(dtest)))




# print("R^2 Score: {:.6f}".format(r2))
print('MAE Valid:', MAE)
# print('MAPE Valid:', MAPE)
print('MBE Valid:', MBE)
print('RMSE Valid:', RMSE)
# print('rRMSE Valid:', rRMSE)
# print('t-stat:', t_stat)



y_true = np.array(y_test, dtype=np.float)
y_pred = np.array(model.predict(dtest), dtype=np.float)

data_prediction = list(zip(y_true,y_pred))
data_prediction = pd.DataFrame(data_prediction, columns=['Test','Prediction'])
data_prediction.head(10)


data_prediction=data_prediction.rename({'Radiation':'Radiation_observed'}, axis=1)
data_prediction.plot(figsize=(20,12))
plt.ylabel('Radiation(W/m2)')
plt.show()


end = time.time()

print(f"Runtime of the program is {end - start}")